In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

### Data Reading

In [0]:
df=spark.read.format("parquet").load("abfss://bronze@databricksaniete.dfs.core.windows.net/orders")
df.limit(10).display()
df.count()

order_id,customer_id,product_id,order_date,quantity,total_amount,_rescued_data
O00001,C00710,P0159,2023-03-22,3,2022.87,null
O00002,C00954,P0036,2023-06-30,2,3560.74,null
O00003,C01578,P0427,2023-11-06,3,5903.52,null
O00004,C00962,P0332,2024-02-27,3,4107.99,null
O00005,C00156,P0038,2024-10-13,5,5784.95,null
O00006,C00521,P0174,2023-05-17,5,407.75,null
O00007,C00982,P0352,2024-01-18,4,4907.64,null
O00008,C00976,P0172,2023-01-10,4,7037.88,null
O00009,C01001,P0238,2023-04-20,3,4076.97,null
O00010,C00702,P0258,2023-07-07,4,5695.64,null


10000

In [0]:
df.printSchema()
df.count()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- _rescued_data: string (nullable = true)



10000

In [0]:
df=df.withColumnRenamed("_rescued_data","rescued_data")

In [0]:
df=df.drop("rescued_data")
df.limit(10).display()
df.count()

order_id,customer_id,product_id,order_date,quantity,total_amount
O00001,C00710,P0159,2023-03-22,3,2022.87
O00002,C00954,P0036,2023-06-30,2,3560.74
O00003,C01578,P0427,2023-11-06,3,5903.52
O00004,C00962,P0332,2024-02-27,3,4107.99
O00005,C00156,P0038,2024-10-13,5,5784.95
O00006,C00521,P0174,2023-05-17,5,407.75
O00007,C00982,P0352,2024-01-18,4,4907.64
O00008,C00976,P0172,2023-01-10,4,7037.88
O00009,C01001,P0238,2023-04-20,3,4076.97
O00010,C00702,P0258,2023-07-07,4,5695.64


10000

In [0]:
df=df.withColumn('order_date',to_timestamp(col('order_date'),'MM-dd-yyyy'))
df.limit(10).display()
df.count()

order_id,customer_id,product_id,order_date,quantity,total_amount
O00001,C00710,P0159,2023-03-22T00:00:00.000Z,3,2022.87
O00002,C00954,P0036,2023-06-30T00:00:00.000Z,2,3560.74
O00003,C01578,P0427,2023-11-06T00:00:00.000Z,3,5903.52
O00004,C00962,P0332,2024-02-27T00:00:00.000Z,3,4107.99
O00005,C00156,P0038,2024-10-13T00:00:00.000Z,5,5784.95
O00006,C00521,P0174,2023-05-17T00:00:00.000Z,5,407.75
O00007,C00982,P0352,2024-01-18T00:00:00.000Z,4,4907.64
O00008,C00976,P0172,2023-01-10T00:00:00.000Z,4,7037.88
O00009,C01001,P0238,2023-04-20T00:00:00.000Z,3,4076.97
O00010,C00702,P0258,2023-07-07T00:00:00.000Z,4,5695.64


10000

In [0]:
df=df.withColumn("year",year(col("order_date")))
df.limit(10).display()
df.count()

order_id,customer_id,product_id,order_date,quantity,total_amount,year
O00001,C00710,P0159,2023-03-22T00:00:00.000Z,3,2022.87,2023
O00002,C00954,P0036,2023-06-30T00:00:00.000Z,2,3560.74,2023
O00003,C01578,P0427,2023-11-06T00:00:00.000Z,3,5903.52,2023
O00004,C00962,P0332,2024-02-27T00:00:00.000Z,3,4107.99,2024
O00005,C00156,P0038,2024-10-13T00:00:00.000Z,5,5784.95,2024
O00006,C00521,P0174,2023-05-17T00:00:00.000Z,5,407.75,2023
O00007,C00982,P0352,2024-01-18T00:00:00.000Z,4,4907.64,2024
O00008,C00976,P0172,2023-01-10T00:00:00.000Z,4,7037.88,2023
O00009,C01001,P0238,2023-04-20T00:00:00.000Z,3,4076.97,2023
O00010,C00702,P0258,2023-07-07T00:00:00.000Z,4,5695.64,2023


10000

In [0]:
df1=df.withColumn('flag',dense_rank().over(Window.partitionBy('year').orderBy(desc('total_amount'))))
df1.limit(10).display()
df1.count()

order_id,customer_id,product_id,order_date,quantity,total_amount,year,flag
O00957,C01449,P0498,2023-10-05T00:00:00.000Z,5,9952.9,2023,1
O01765,C01515,P0498,2023-05-11T00:00:00.000Z,5,9952.9,2023,1
O03502,C00805,P0498,2023-09-24T00:00:00.000Z,5,9952.9,2023,1
O03660,C01001,P0498,2023-10-08T00:00:00.000Z,5,9952.9,2023,1
O06790,C01819,P0498,2023-02-27T00:00:00.000Z,5,9952.9,2023,1
O03989,C00631,P0440,2023-11-20T00:00:00.000Z,5,9916.25,2023,2
O07763,C00471,P0440,2023-04-12T00:00:00.000Z,5,9916.25,2023,2
O03272,C01879,P0165,2023-02-15T00:00:00.000Z,5,9858.85,2023,3
O03746,C01713,P0165,2023-10-26T00:00:00.000Z,5,9858.85,2023,3
O08261,C00988,P0165,2023-11-29T00:00:00.000Z,5,9858.85,2023,3


10000

In [0]:
df1=df1.withColumn('rank_flag',rank().over(Window.partitionBy('year').orderBy(desc('total_amount'))))
df1.limit(10).display()
df1.count()

order_id,customer_id,product_id,order_date,quantity,total_amount,year,flag,rank_flag
O00957,C01449,P0498,2023-10-05T00:00:00.000Z,5,9952.9,2023,1,1
O01765,C01515,P0498,2023-05-11T00:00:00.000Z,5,9952.9,2023,1,1
O03502,C00805,P0498,2023-09-24T00:00:00.000Z,5,9952.9,2023,1,1
O03660,C01001,P0498,2023-10-08T00:00:00.000Z,5,9952.9,2023,1,1
O06790,C01819,P0498,2023-02-27T00:00:00.000Z,5,9952.9,2023,1,1
O03989,C00631,P0440,2023-11-20T00:00:00.000Z,5,9916.25,2023,2,6
O07763,C00471,P0440,2023-04-12T00:00:00.000Z,5,9916.25,2023,2,6
O03272,C01879,P0165,2023-02-15T00:00:00.000Z,5,9858.85,2023,3,8
O03746,C01713,P0165,2023-10-26T00:00:00.000Z,5,9858.85,2023,3,8
O08261,C00988,P0165,2023-11-29T00:00:00.000Z,5,9858.85,2023,3,8


10000

In [0]:
df1=df1.withColumn('row_flag',row_number().over(Window.partitionBy('year').orderBy(desc('total_amount'))))
df1.limit(10).display()
df1.count()

order_id,customer_id,product_id,order_date,quantity,total_amount,year,flag,rank_flag,row_flag
O00957,C01449,P0498,2023-10-05T00:00:00.000Z,5,9952.9,2023,1,1,1
O01765,C01515,P0498,2023-05-11T00:00:00.000Z,5,9952.9,2023,1,1,2
O03502,C00805,P0498,2023-09-24T00:00:00.000Z,5,9952.9,2023,1,1,3
O03660,C01001,P0498,2023-10-08T00:00:00.000Z,5,9952.9,2023,1,1,4
O06790,C01819,P0498,2023-02-27T00:00:00.000Z,5,9952.9,2023,1,1,5
O03989,C00631,P0440,2023-11-20T00:00:00.000Z,5,9916.25,2023,2,6,6
O07763,C00471,P0440,2023-04-12T00:00:00.000Z,5,9916.25,2023,2,6,7
O03272,C01879,P0165,2023-02-15T00:00:00.000Z,5,9858.85,2023,3,8,8
O03746,C01713,P0165,2023-10-26T00:00:00.000Z,5,9858.85,2023,3,8,9
O08261,C00988,P0165,2023-11-29T00:00:00.000Z,5,9858.85,2023,3,8,10


10000

### **Classes - OOP**

In [0]:
class windows:
    def dense_rank(self,df):
        df_dense_rank=df.withColumn("flag",dense_rank().over(Window.partitionBy('year').orderBy(desc('total_amount'))))
        return df_dense_rank
    def rank(self,df):
        df_rank=df.withColumn("rank_flag",rank().over(Window.partitionBy('year').orderBy(desc('total_amount'))))
        return df_rank
    def row_number(self,df):
        df_row_number=df.withColumn("row_flag",row_number().over(Window.partitionBy('year').orderBy(desc('total_amount'))))
        return df_row_number

In [0]:
df_new=df
obj=windows()
obj.dense_rank(df_new).limit(10).display()

order_id,customer_id,product_id,order_date,quantity,total_amount,year,flag
O00957,C01449,P0498,2023-10-05T00:00:00.000Z,5,9952.9,2023,1
O01765,C01515,P0498,2023-05-11T00:00:00.000Z,5,9952.9,2023,1
O03502,C00805,P0498,2023-09-24T00:00:00.000Z,5,9952.9,2023,1
O03660,C01001,P0498,2023-10-08T00:00:00.000Z,5,9952.9,2023,1
O06790,C01819,P0498,2023-02-27T00:00:00.000Z,5,9952.9,2023,1
O03989,C00631,P0440,2023-11-20T00:00:00.000Z,5,9916.25,2023,2
O07763,C00471,P0440,2023-04-12T00:00:00.000Z,5,9916.25,2023,2
O03272,C01879,P0165,2023-02-15T00:00:00.000Z,5,9858.85,2023,3
O03746,C01713,P0165,2023-10-26T00:00:00.000Z,5,9858.85,2023,3
O08261,C00988,P0165,2023-11-29T00:00:00.000Z,5,9858.85,2023,3


### **Data Writing**

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@databricksaniete.dfs.core.windows.net/orders")

In [0]:
%sql
CREATE TABLE if not EXISTS databricks_cata.silver.orders_silver
USING DELTA
LOCATION 'abfss://silver@databricksaniete.dfs.core.windows.net/orders'